In [23]:
import sys
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
%reload_ext memory_profiler


In [18]:
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
from MKLpy.metrics import margin, radius, ratio, trace, frobenius
from MKLpy.preprocessing import kernel_normalization
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from MKLpy.multiclass import OneVsRestMKLClassifier, OneVsOneMKLClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [28]:
from mkl_data_processing import storage_location,cross_validation_results_location
from mkl_model_cross_validation import load_pickled_in_filename

In [19]:
import gc #for garbage collection

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
import fileutils
import psutil
import pickle
import fnmatch
import os
import torch
import time
import clfutils
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, hamming_loss
import new_alternate_single_svm as nalsvm

In [21]:
KL_rbf = generators.RBF_generator(X_train, gamma=[.001, .01, .1])

NameError: name 'X_train' is not defined

In [4]:
def load_pickled_in_filename(file):
    # load a simple pickled file and return it. its a bit different to the method used for the dictionary as this
    # is pure Python 3.x
    pickle_in = open(file, 'rb')
    return pickle.load(pickle_in)


In [3]:
import os
import pickle
file_location = '/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CCL.L'
files_pkl = os.listdir(file_location)

In [15]:
file_to_be_loaded = os.path.join(file_location, files_pkl[3])

In [16]:
load_pickled_in_filename(file_to_be_loaded)

{'20170704': [(1, 100, [1.0, 1.0, 0.32420091324200917])],
 '20170705': [(1,
   100,
   [0.9628099173553719, 0.9813278008298755, 0.8921161825726142])],
 '20170711': [(1,
   100,
   [0.7952127659574468, 0.9533333333333333, 0.9986666666666666])],
 '20170818': [(1,
   100,
   [0.858974358974359, 0.41025641025641024, 0.3137614678899082])]}

In [25]:
 for symbol in ['ECM.L','SHP.L']:
        print(symbol) # which symbol - unnecessary at this point
        cross_validation_data_location = cross_validation_results_location(symbol)
        print(cross_validation_data_location)
        #clean_data_location = storage_location(symbol)

ECM.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/ECM.L
SHP.L
/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/SHP.L


In [29]:
pkl_files_cross_val = os.listdir(cross_validation_data_location)

In [36]:
idx =1
loaded_pickle =load_pickled_in_filename(os.path.join(cross_validation_data_location,pkl_files_cross_val[idx]))

In [51]:
loaded_pickle.columns.values

array([('ECM.L', '20170705', 'LabelsAlternateFive'),
       ('ECM.L', '20170710', 'LabelsAlternateFive'),
       ('ECM.L', '20170711', 'LabelsAlternateFive'),
       ('ECM.L', '20170712', 'LabelsAlternateFive'),
       ('ECM.L', '20170717', 'LabelsAlternateFive'),
       ('ECM.L', '20170718', 'LabelsAlternateFive'),
       ('ECM.L', '20170719', 'LabelsAlternateFive'),
       ('ECM.L', '20170720', 'LabelsAlternateFive'),
       ('ECM.L', '20170725', 'LabelsAlternateFive'),
       ('ECM.L', '20170726', 'LabelsAlternateFive'),
       ('ECM.L', '20170728', 'LabelsAlternateFive'),
       ('ECM.L', '20170802', 'LabelsAlternateFive'),
       ('ECM.L', '20170803', 'LabelsAlternateFive'),
       ('ECM.L', '20170807', 'LabelsAlternateFive'),
       ('ECM.L', '20170808', 'LabelsAlternateFive'),
       ('ECM.L', '20170809', 'LabelsAlternateFive'),
       ('ECM.L', '20170810', 'LabelsAlternateFive'),
       ('ECM.L', '20170811', 'LabelsAlternateFive'),
       ('ECM.L', '20170814', 'LabelsAlternateF

In [47]:
loaded_dict= loaded_pickle.to_dict()

In [58]:
keys =list(loaded_dict.keys())

In [61]:
symbol = keys[0][0]
date = keys[0][1]
label_type = keys[0][2]

In [74]:
loaded_pickle[keys[0]][0].values[0][1]

0.9920948616600791

In [76]:
 #{k1 + k2 : v for k1, v1 in d.iteritems() for k2, v in v1.iteritems() } 
example_dict = {k1 + k2 : v for k1, v1 in loaded_pickle.iteritems() for k2, v in v1.iteritems() } 

In [78]:
list(example_dict.keys())

[('ECM.L', '20170705', 'LabelsAlternateFive', 0.0, 0.01),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.0, 1.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.0, 10.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.0, 100.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.1, 0.01),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.1, 1.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.1, 10.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.1, 100.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.2, 0.01),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.2, 1.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.2, 10.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 0.2, 100.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 1.0, 0.01),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 1.0, 1.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 1.0, 10.0),
 ('ECM.L', '20170705', 'LabelsAlternateFive', 1.0, 100.0),
 ('ECM.L', '20170710', 'LabelsAlternateFive', 0.0, 0.01),
 ('ECM.L', '20

In [80]:
list(example_dict.values())

[[0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.9920948616600791],
 [0.9881889763779528, 0.9920948616600791, 0.99209486166